In [1]:
# import libraries
import numpy as np
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [2]:
# a helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Read training and test sets, sort train set by session start time.**

In [4]:
train_df = pd.read_csv('./data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('./data//test_sessions.csv',
                      index_col='session_id')

# Convert time1, ..., time10 columns to datetime type
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


**Transform data into format which can be fed into `CountVectorizer`**

In [7]:
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites].fillna(0).astype('int').to_csv('train_sessions_text.txt', 
                                               sep=' ', 
                       index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv('test_sessions_text.txt', 
                                              sep=' ', 
                       index=None, header=None)

In [8]:
!head -5 train_sessions_text.txt

'head' is not recognized as an internal or external command,
operable program or batch file.


**Fit `CountVectorizer` and trasfrom data with it.**

In [12]:
%%time
cv = CountVectorizer()
with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)
print(X_train.shape, X_test.shape)

(253561, 41592) (82797, 41592)
CPU times: total: 2.16 s
Wall time: 2.81 s


**Save train targets into a separate vector.**

In [13]:
y_train = train_df['target'].astype('int')

### train Logistic regression

In [15]:
logit = LogisticRegression(C=1, random_state=17)

In [19]:
%%time
cv_scores = cross_val_score(logit, X_train, y_train, cv=5, scoring='roc_auc')

CPU times: total: 1.33 s
Wall time: 4.3 s


In [23]:
cv_scores

array([0.91603556, 0.8431174 , 0.87907924, 0.89093035, 0.91099186])

In [24]:
cv_scores.mean()

np.float64(0.8880308813650295)

In [25]:
%%time
logit.fit(X_train, y_train)

CPU times: total: 156 ms
Wall time: 991 ms


LogisticRegression(C=1, random_state=17)

In [26]:
test_pred_logit1 = logit.predict_proba(X_test)[:, 1]

In [27]:
test_pred_logit1

array([3.49637074e-03, 4.94332373e-11, 5.66205295e-12, ...,
       1.00328833e-02, 4.05119249e-04, 1.37432056e-06], shape=(82797,))

In [28]:
## CV 0.885
write_to_submission_file(test_pred_logit1, 'logit_subm1.txt') ## .908 ROC AUC Public LB

### Time Features
 - hour when the session started
 - morning 
 - day
 - eve
 - night

In [29]:
def add_time_features(time1_series, X_sparse):
    hour = time1_series.apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [30]:
test_df.loc[:, 'time1'].fillna(0).apply(lambda ts: ts.hour).head()

session_id
1    11
2    11
3    15
4    10
5    15
Name: time1, dtype: int64

In [31]:
%%time
X_train_with_time = add_time_features(train_df['time1'].fillna(0), X_train)
X_test_with_time = add_time_features(test_df['time1'].fillna(0), X_test)

CPU times: total: 266 ms
Wall time: 604 ms


In [32]:
X_train_with_time.shape, X_test_with_time.shape

((253561, 41596), (82797, 41596))

In [53]:
%%time
cv_scores = cross_val_score(logit, X_train_with_time, y_train, cv=5, scoring='roc_auc')

CPU times: user 2min 12s, sys: 3.26 s, total: 2min 16s
Wall time: 22.9 s


In [54]:
cv_scores

array([0.92516354, 0.90530702, 0.93167349, 0.943663  , 0.94800673])

In [55]:
cv_scores.mean()

0.930762754389869

In [34]:
%%time
logit.fit(X_train_with_time, y_train)

CPU times: total: 312 ms
Wall time: 1.19 s


LogisticRegression(C=1, random_state=17)

In [35]:
test_pred_logit2 = logit.predict_proba(X_test_with_time)[:, 1]

In [36]:
test_pred_logit2

array([9.48623795e-05, 1.51907103e-08, 1.52101138e-10, ...,
       3.19570780e-04, 1.19102031e-05, 9.30722766e-08], shape=(82797,))

In [37]:
## CV .9307
write_to_submission_file(test_pred_logit2, 'logit_subm2.txt') ## ROC AUC 0.93565 Public LB